In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
import numpy as np
import pandas as pd
np.set_printoptions(suppress=True)


In [31]:
import warnings
warnings.filterwarnings('ignore')

# Datos de ejemplo de visualizaciones de video


---



In [2]:
views = pd.DataFrame([1295., 25., 19000., 5., 1., 300.], columns=['views'])
views

,views
0,1295.0
1,25.0
2,19000.0
3,5.0
4,1.0
5,300.0


# Escalador estandar $\frac{x_i - \mu}{\sigma}$ $\mu$: valor medio, $\sigma$: desviación estándar

In [3]:
ss = StandardScaler()
views['zscore'] = ss.fit_transform(views[['views']])
views

,views,zscore
0,1295.0,-0.307214
1,25.0,-0.489306
2,19000.0,2.231317
3,5.0,-0.492173
4,1.0,-0.492747
5,300.0,-0.449877


In [4]:
vw = np.array(views['views'])
(vw - np.mean(vw)) / np.std(vw)

array([-0.30721413, -0.4893059 ,  2.23131709, -0.49217348, -0.492747  ,
       -0.44987658])

# Escalador Min-Max $\frac{x_i - min(x)}{max(x) - min(x)}$

---



In [5]:
mms = MinMaxScaler()
views['minmax'] = mms.fit_transform(views[['views']])
views

,views,zscore,minmax
0,1295.0,-0.307214,0.068109
1,25.0,-0.489306,0.001263
2,19000.0,2.231317,1.000000
3,5.0,-0.492173,0.000211
4,1.0,-0.492747,0.000000
5,300.0,-0.449877,0.015738


In [6]:
(vw - np.min(vw)) / (np.max(vw) - np.min(vw))

array([0.06810885, 0.00126322, 1.        , 0.00021054, 0.        ,
       0.01573767])

# Escalador Robusto  $\frac{x_i - mediana(x)}{IQR_{(1,3)}(x)}$

In [7]:
rs = RobustScaler()
views['robust'] = rs.fit_transform(views[['views']])
views

,views,zscore,minmax,robust
0,1295.0,-0.307214,0.068109,1.092883
1,25.0,-0.489306,0.001263,-0.132690
2,19000.0,2.231317,1.000000,18.178528
3,5.0,-0.492173,0.000211,-0.151990
4,1.0,-0.492747,0.000000,-0.155850
5,300.0,-0.449877,0.015738,0.132690


In [8]:
quartiles = np.percentile(vw, (25., 75.))
iqr = quartiles[1] - quartiles[0]
(vw - np.median(vw)) / iqr

array([ 1.09288299, -0.13268999, 18.17852835, -0.15199035, -0.15585042,
        0.13268999])

# Selección de atributos

In [9]:
import numpy as np
import pandas as pd
np.set_printoptions(suppress=True)
pt = np.get_printoptions()['threshold']

# Métodos basados en umbral




Se trata de una estrategia de selección de características basada en un filtro, en la que se puede utilizar algún tipo de corte o umbral para
limitar el número total de características durante la selección. Los umbrales 
pueden utilizarse durante el propio proceso de ingeniería de características, donde se pueden especificar parámetros de umbral.
Un ejemplo sencillo sería limitar los términos de las características en el modelo Bag of Words.

Scikit-learn proporciona parámetros como min_df y max_
df que pueden utilizarse para especificar los umbrales para ignorar los términos que tienen una frecuencia de documento superior o inferior a los umbrales especificados por el usuario.


## Limitando características en el modelo *bag of words*

In [10]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(min_df=0.1, max_df=0.85, max_features=2000)
cv

CountVectorizer(max_df=0.85, max_features=2000, min_df=0.1)

Esto básicamente construye un vectorizador de conteo que ignora los términos ocurren en menos del 10% del
total del corpus y también ignora los términos que aparecen en más del 85% del total. Además, también
un límite de 2000 características máximas en el conjunto de características.

## Umbral basado en varianza

En este método se eliminan las características que tienen una varianza baja (por debajo del umbral especificado por el usuario).

Esto significa que queremos eliminar las características que
tienen valores más o menos constantes en todas las observaciones de nuestros conjuntos de datos. Podemos aplicar esto a
nuestro conjunto de datos de Pokémon, que hemos utilizado anteriormente en este capítulo. Primero convertimos la característica Generación en una característica
categórica de la siguiente manera.


In [11]:
df = pd.read_csv('Pokemon.csv')
poke_gen = pd.get_dummies(df['Generation'])
poke_gen.head()

,Gen 1,Gen 2,Gen 3,Gen 4,Gen 5,Gen 6
0,True,False,False,False,False,False
1,True,False,False,False,False,False
2,True,False,False,False,False,False
3,True,False,False,False,False,False
4,True,False,False,False,False,False


A continuación, queremos eliminar las características en las que la varianza es inferior a 0.15

In [12]:
from sklearn.feature_selection import VarianceThreshold

vt = VarianceThreshold(threshold=.15)
vt.fit(poke_gen)

VarianceThreshold(threshold=0.15)

Para ver las varianzas, así como las características que fueron finalmente seleccionadas por este algoritmo, podemos utilizar la propiedad
variances_ y la función get_support(...) respectivamente.

In [13]:
pd.DataFrame({'variance': vt.variances_,
              'select_feature': vt.get_support()},
            index=poke_gen.columns).T

,Gen 1,Gen 2,Gen 3,Gen 4,Gen 5,Gen 6
variance,0.164444,0.114944,0.16,0.128373,0.163711,0.091994
select_feature,True,False,True,False,True,False


In [14]:
poke_gen_subset = poke_gen.iloc[:,vt.get_support()].head()
poke_gen_subset

,Gen 1,Gen 3,Gen 5
0,True,False,False
1,True,False,False
2,True,False,False
3,True,False,False
4,True,False,False


# Métodos estadísticos

Otro método de selección de características basado en un filtro, que es ligeramente más sofisticado, consiste en
seleccionar características basadas en pruebas estadísticas univariadas. Se pueden utilizar varias pruebas estadísticas para los modelos de regresión y clasificación, como la información mutua, el ANOVA (análisis de varianza) y las pruebas de chi-cuadrado. A partir de las puntuaciones obtenidas en estas pruebas estadísticas, se pueden seleccionar las mejores características en función de su puntuación.

Carguemos ahora un conjunto de datos de muestra con 30 características. Este conjunto de datos se conoce como Wisconsin
Diagnostic Breast Cancer, que también está disponible en su formato nativo o crudo en https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic), que es el repositorio de aprendizaje automático de la UCI.

Utilizaremos scikit-learn para cargar las características de los datos y la variable de clase de respuesta.


In [15]:
from sklearn.datasets import load_breast_cancer

bc_data = load_breast_cancer()
bc_features = pd.DataFrame(bc_data.data, columns=bc_data.feature_names)
bc_classes = pd.DataFrame(bc_data.target, columns=['IsMalignant'])

# build featureset and response class labels 
bc_X = np.array(bc_features)
bc_y = np.array(bc_classes).T[0]
print('Feature set shape:', bc_X.shape)
print('Response class shape:', bc_y.shape)

Feature set shape: (569, 30)
Response class shape: (569,)


In [16]:
np.set_printoptions(threshold=30)
print('Feature set data [shape: '+str(bc_X.shape)+']')
print(np.round(bc_X, 2), '\n')
print('Feature names:')
print(np.array(bc_features.columns), '\n')
print('Predictor Class label data [shape: '+str(bc_y.shape)+']')
print(bc_y, '\n')
print('Predictor name:', np.array(bc_classes.columns))
np.set_printoptions(threshold=pt)

Feature set data [shape: (569, 30)]
[[ 17.99  10.38 122.8  ...   0.27   0.46   0.12]
 [ 20.57  17.77 132.9  ...   0.19   0.28   0.09]
 [ 19.69  21.25 130.   ...   0.24   0.36   0.09]
 ...
 [ 16.6   28.08 108.3  ...   0.14   0.22   0.08]
 [ 20.6   29.33 140.1  ...   0.26   0.41   0.12]
 [  7.76  24.54  47.92 ...   0.     0.29   0.07]] 

Feature names:
['mean radius' 'mean texture' 'mean perimeter' 'mean area'
 'mean smoothness' 'mean compactness' 'mean concavity'
 'mean concave points' 'mean symmetry' 'mean fractal dimension'
 'radius error' 'texture error' 'perimeter error' 'area error'
 'smoothness error' 'compactness error' 'concavity error'
 'concave points error' 'symmetry error' 'fractal dimension error'
 'worst radius' 'worst texture' 'worst perimeter' 'worst area'
 'worst smoothness' 'worst compactness' 'worst concavity'
 'worst concave points' 'worst symmetry' 'worst fractal dimension'] 

Predictor Class label data [shape: (569,)]
[0 0 0 ... 0 0 1] 

Predictor name: ['IsMaligna

Esto nos da una mejor perspectiva de los datos que estamos tratando. La variable objetivo es una clase binaria
donde 1 indica que el tumor detectado era benigno y 0 que era maligno. También podemos ver
las 30 características que son números reales que describen las características de los núcleos celulares presentes en las imágenes digitalizadas de la masa mamaria.
Utilicemos ahora la prueba de chi-cuadrado en este conjunto de características y seleccionemos las 15 mejores características

In [17]:
from sklearn.feature_selection import chi2, SelectKBest

skb = SelectKBest(score_func=chi2, k=15)
skb.fit(bc_X, bc_y)

SelectKBest(k=15, score_func=<function chi2 at 0x000001A07B297A60>)

Se puede ver que hemos pasado nuestras características de entrada (bc_X) y las correspondientes salidas de la clase objetivo
(bc_y) a la función fit(...) al calcular las métricas necesarias. La prueba de chi-cuadrado calculará
estadísticas entre cada característica y la variable de clase (pruebas univariadas). Seleccionar las K principales características elimina las que tengan una puntuación baja y, en consecuencia, poca correlación con la variable objetivo. Por lo tanto, no son útiles para construir modelos. 
Clasificamos las puntuaciones para ver las características más relevantes
relevantes:

In [18]:
feature_scores = [(item, score) for item, score in zip(bc_data.feature_names, skb.scores_)]
sorted(feature_scores, key=lambda x: -x[1])[:10]

[('worst area', 112598.43156405364),
 ('mean area', 53991.65592375085),
 ('area error', 8758.504705334477),
 ('worst perimeter', 3665.0354163405946),
 ('mean perimeter', 2011.1028637679074),
 ('worst radius', 491.6891574333232),
 ('mean radius', 266.104917195178),
 ('perimeter error', 250.571896359822),
 ('worst texture', 174.44939960571074),
 ('mean texture', 93.8975080986333)]

In [19]:
select_features_kbest = skb.get_support()
feature_names_kbest = bc_data.feature_names[select_features_kbest]
feature_subset_df = bc_features[feature_names_kbest]
bc_SX = np.array(feature_subset_df)
print(bc_SX.shape)
print(feature_names_kbest)

(569, 15)
['mean radius' 'mean texture' 'mean perimeter' 'mean area'
 'mean concavity' 'radius error' 'perimeter error' 'area error'
 'worst radius' 'worst texture' 'worst perimeter' 'worst area'
 'worst compactness' 'worst concavity' 'worst concave points']


In [20]:
np.round(feature_subset_df.iloc[20:25], 2)

,mean radius,mean texture,mean perimeter,mean area,mean concavity,radius error,perimeter error,area error,worst radius,worst texture,worst perimeter,worst area,worst compactness,worst concavity,worst concave points
20,13.08,15.71,85.63,520.0,0.05,0.19,1.38,14.67,14.50,20.49,96.09,630.5,0.28,0.19,0.07
21,9.50,12.44,60.34,273.9,0.03,0.28,1.91,15.70,10.23,15.66,65.13,314.9,0.11,0.09,0.06
22,15.34,14.26,102.50,704.4,0.21,0.44,3.38,44.91,18.07,19.08,125.10,980.9,0.60,0.63,0.24
23,21.16,23.04,137.20,1404.0,0.11,0.69,4.30,93.99,29.17,35.59,188.00,2615.0,0.26,0.32,0.20
24,16.65,21.38,110.00,904.6,0.15,0.81,5.46,102.60,26.46,31.56,177.00,2215.0,0.36,0.47,0.21


Construyamos ahora un sencillo
modelo de clasificación utilizando la regresión logística en el conjunto original de 30 características y comparemos el
con otro modelo construido con las 15 características seleccionadas. Para la evaluación del modelo,
utilizaremos la métrica de precisión (porcentaje de predicciones correctas)

In [34]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

lr = LogisticRegression( random_state=42)

full_feat_acc = np.average(cross_val_score(lr, bc_X, bc_y, scoring='accuracy', cv=5))

sel_feat_acc = np.average(cross_val_score(lr, bc_SX, bc_y, scoring='accuracy', cv=5))

print('Model accuracy statistics with 5-fold cross validation')
print('Model accuracy with complete feature set', bc_X.shape, ':', full_feat_acc)
print('Model accuracy with selected feature set', bc_SX.shape, ':', sel_feat_acc)

Model accuracy statistics with 5-fold cross validation
Model accuracy with complete feature set (569, 30) : 0.9455364073901567
Model accuracy with selected feature set (569, 15) : 0.9420431609998448


# Eliminación recursiva de características




También se pueden clasificar y puntuar las características con la ayuda de un modelo de *machine learning*, de manera que
se sigan eliminando recursivamente las características con menor puntuación hasta llegar al subconjunto de características específicas.
La eliminación recursiva de características, también conocida como RFE, es una técnica popular de método envolvente. La idea básica es comenzar con un estimador específico de ML
como el algoritmo de Regresión Logística que utilizamos para la clasificación. A continuación, tomamos las 30 características y lavariable objetivo. RFE asigna pesos a estas características
basándose en el ajuste del modelo. Las características con los pesos más pequeños se eliminan y luego se ajusta un modelo de nuevo.

Este proceso se lleva a cabo recursivamente varias veces y cada vez se eliminan las características con las puntuaciones/pesos más bajos, hasta que el subconjunto de características podado contenga el número deseado de características que el usuario quería seleccionar (esto se toma como parámetro de entrada al
al principio). Esta estrategia también se conoce popularmente como *backward elimination*. 

In [32]:
from sklearn.feature_selection import RFE

lr = LogisticRegression()
rfe = RFE(estimator=lr, n_features_to_select=15, step=1)
rfe.fit(bc_X, bc_y)

RFE(estimator=LogisticRegression(), n_features_to_select=15)

In [23]:
select_features_rfe = rfe.get_support()
feature_names_rfe = bc_data.feature_names[select_features_rfe]
print(feature_names_rfe)

['mean radius' 'mean texture' 'mean compactness' 'mean concavity'
 'mean concave points' 'mean symmetry' 'texture error' 'perimeter error'
 'worst radius' 'worst texture' 'worst compactness' 'worst concavity'
 'worst concave points' 'worst symmetry' 'worst fractal dimension']


¿Podemos comparar este subconjunto de características con el que obtuvimos mediante pruebas estadísticas y ver qué características son comunes entre ambos subconjuntos?
Utilizamos las operaciones de conjunto
para obtener la lista de características seleccionadas por ambas técnicas.

In [ ]:
set(feature_names_kbest) & set(feature_names_rfe) & set(feature_impo)

{'mean concavity',
 'mean radius',
 'mean texture',
 'perimeter error',
 'worst compactness',
 'worst concave points',
 'worst concavity',
 'worst radius',
 'worst texture'}

In [36]:
lista=[]
for v in feature_importances:
    lista.append(v[0])
print(lista)

['mean radius', 'mean texture', 'mean perimeter', 'mean area', 'mean smoothness', 'mean compactness', 'mean concavity', 'mean concave points', 'mean symmetry', 'mean fractal dimension', 'radius error', 'texture error', 'perimeter error', 'area error', 'smoothness error', 'compactness error', 'concavity error', 'concave points error', 'symmetry error', 'fractal dimension error', 'worst radius', 'worst texture', 'worst perimeter', 'worst area', 'worst smoothness', 'worst compactness', 'worst concavity', 'worst concave points', 'worst symmetry', 'worst fractal dimension']


# Selección basada en modelos


Los árboles de decisión y *random forests* pueden
utilizarse no sólo para el modelado, sino también para la selección de características. Pueden utilizarse para calcular la importancia de cada
características, seleccionar las mejores y
descartar las irrelevantes con puntuaciones más bajas

In [25]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier()
rfc.fit(bc_X, bc_y)

RandomForestClassifier()

In [26]:
importance_scores = rfc.feature_importances_
feature_importances = [(feature, score) for feature, score in zip(bc_data.feature_names, importance_scores)]
sorted(feature_importances, key=lambda x: -x[1])[:10]

[('worst concave points', 0.15720675600399014),
 ('worst perimeter', 0.14402114947663353),
 ('worst radius', 0.13184827483991657),
 ('worst area', 0.09397236431000036),
 ('mean concave points', 0.09368299151141844),
 ('mean area', 0.0464639817172492),
 ('mean concavity', 0.04543240079881447),
 ('area error', 0.038303769926483204),
 ('mean perimeter', 0.03783678678027513),
 ('worst concavity', 0.029127396151689956)]

Ahora podemos filtrar las n características principales según sea necesario 

Ejercicio: Encontrar cuántas de las características mejor clasificadas del modelo de *Random Forest* son comunes con los dos selectores de características anteriores

# Reducción de dimensionalidad

In [27]:
from sklearn.decomposition import PCA
pca = PCA(n_components=3)
pca.fit(bc_X)

PCA(n_components=3)

In [28]:
pca.explained_variance_ratio_

array([0.98204467, 0.01617649, 0.00155751])

In [29]:
bc_pca = pca.transform(bc_X)
np.round(bc_pca, 2)

array([[1160.14, -293.92,   48.58],
       [1269.12,   15.63,  -35.39],
       [ 995.79,   39.16,   -1.71],
       ...,
       [ 314.5 ,   47.55,  -10.44],
       [1124.86,   34.13,  -19.74],
       [-771.53,  -88.64,   23.89]])

Construyamos un modelo de regresión logística como antes y utilicemos la precisión del modelo para evaluar la calidad, utilizando sólo tres características.

In [30]:
np.average(cross_val_score(lr, bc_pca, bc_y, scoring='accuracy', cv=5))

0.9262071106970968

Podemos ver en el resultado anterior que, a pesar de utilizar sólo tres características derivadas de los
componentes principales en lugar de las 30 características originales, seguimos obteniendo una precisión del modelo cercana al 93%,
lo que es bastante decente.